In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('')

### Вытаскиваем уникальные значения из некоторых категориальных фичей¶

In [ ]:
#это метод OHE.
def convert_lists(df: pd.DataFrame) -> pd.DataFrame:
    cols = [
        "Balcony", "Bathroom", "Windows", "Sale_method",
        "House_type"
           ]
    new_cols = []
    for col in cols:
        series = df.loc[:, col]
        data = []
        unique_values = set()
        new_cols_data = []

        for row in series.values:
            row_c = [value.strip() for value in (row.split(",") if isinstance(row, str) else [])]
            for element in row_c:
                if not element in unique_values:
                    unique_values.add(element)
            data.append(row_c)
    
        unique_values = list(unique_values)

        for row in data:
            new_cols_data.append([1 if value in row else 0 for value in unique_values])
    
        col_names = [f"{col}_{i+1}" for i in range(len(unique_values))]
        new_cols.append(pd.DataFrame(data=new_cols_data, columns=col_names, dtype=np.int8))
        print("\n".join(map(str,zip(unique_values, col_names))))

        df.drop(columns=[col], axis=1, inplace=True)

    return pd.concat([df]+new_cols, axis=1)

df = convert_lists(df)